In [5]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python import keras as kt
# RNN imports
import keras
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN, SimpleRNN
from keras.utils import to_categorical

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Using TensorFlow backend.


In [6]:

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


### Shape of data

In [7]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 25, 1000)
Test data shape: (443, 25, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [8]:
# prep datasets for RNN

# fomat data to (batch_size, timesteps, input_dim)
k_X_train = np.swapaxes(X_train_valid, 1, 2)
k_X_test = np.swapaxes(X_test, 1, 2)
print("Swapped axes:",k_X_train.shape, k_X_test.shape)

# remove VOG channels (23-25)
k_X_train = k_X_train[: , : , :22 ]
k_X_test = k_X_test[: , : , :22 ]
print("Removed VOG channels:", k_X_train.shape, k_X_test.shape)

# encode output labels
print("Raw labels:", y_train_valid[0:10])
k_y_train = y_train_valid - 769
k_y_test = y_test - 769
print("Fixed:", k_y_train[0:10])
k_y_train_categ = to_categorical(k_y_train, 4)
k_y_test_categ = to_categorical(k_y_test, 4)
print("Categorical one-hot encoding:\n",k_y_train_categ[0:3])

Swapped axes: (2115, 1000, 25) (443, 1000, 25)
Removed VOG channels: (2115, 1000, 22) (443, 1000, 22)
Raw labels: [771 772 769 769 769 769 771 770 772 772]
Fixed: [2 3 0 0 0 0 2 1 3 3]
Categorical one-hot encoding:
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]


In [11]:
# RNN model parameters
k_hidden_size = 100
k_input_shape = k_X_train.shape[1:]
hidden_sizes = [50, 100, 200]
learn_rates = [0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
decays = [0.0, 0.005, 0.05, .1]
l1_regs = [0.0, 0.0001, 0.001, 0.1]
activations = ['relu', 'tanh']

for hids in hidden_sizes:
    for lrt in learn_rates:
        for dcy in decays:
            for reg in l1_regs:
                for actv in activations:
                    # RNN model
                    model = Sequential()
                    model.add(SimpleRNN(hids, input_shape=k_input_shape, activation=actv, recurrent_regularizer=keras.regularizers.l1(reg), dropout=0.5, recurrent_dropout=0.5))
                    model.add(Dense(4, activation='softmax'))

                    # Optimizer
                    adam = keras.optimizers.Adam(lr=lrt, beta_1=0.9, beta_2=0.999, epsilon=None, decay=dcy, amsgrad=False)
                    model.compile(loss = 'categorical_crossentropy', 
                                  optimizer=adam, 
                                  metrics=['accuracy'],
                                 )

                    # Train the model
                    history = model.fit(k_X_train, k_y_train_categ, epochs=3, verbose=0, validation_split=0.1)
                    
                    # print out validation accuracy and params
                    print(history.history['val_acc'])
                    print("Val acc:", history.history['val_acc'][-1], "hidden size:", hids, "learn rate:", lrt, "adam decay:", dcy, "l1reg:", reg, "activation:", actv)



# # Plot training & validation accuracy values
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

[0.26886792509060986, 0.26886792509060986, 0.26886792509060986]
Val acc: 0.26886792509060986 hidden size: 50 learn rate: 0.05 adam decay: 0.0 l1reg: 0.0 activation: relu
[0.26886792565291784, 0.18396226429152038, 0.2641509439585344]
Val acc: 0.2641509439585344 hidden size: 50 learn rate: 0.05 adam decay: 0.0 l1reg: 0.0 activation: tanh
[0.26886792509060986, 0.26886792509060986, 0.26886792509060986]
Val acc: 0.26886792509060986 hidden size: 50 learn rate: 0.05 adam decay: 0.0 l1reg: 0.0001 activation: relu
[0.22641509448019964, 0.240566038860465, 0.27358490622268533]
Val acc: 0.27358490622268533 hidden size: 50 learn rate: 0.05 adam decay: 0.0 l1reg: 0.0001 activation: tanh
[0.26886792509060986, 0.26886792509060986, 0.26886792509060986]
Val acc: 0.26886792509060986 hidden size: 50 learn rate: 0.05 adam decay: 0.0 l1reg: 0.001 activation: relu
[0.24528301914907852, 0.31603773641136457, 0.2830188679245283]
Val acc: 0.2830188679245283 hidden size: 50 learn rate: 0.05 adam decay: 0.0 l1reg:

KeyboardInterrupt: 

In [25]:

# evaluate

model.evaluate(k_X_test, k_y_test_categ, verbose=1)

443/443 [==============================] - 1s 2ms/step


[1.4219469386352643, 0.23250564300448845]

# That is disappointing